In [14]:
from utils.load_results import *
from utils.plot_helpers import *

import pandas as pd
#import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('default')
import torch
from utils.analysis_from_interaction import *
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim
import os
if not os.path.exists('analysis'):
    os.makedirs('analysis')
#import plotly.express as px

In [15]:
datasets = ['(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)']
n_values = [4, 8, 16, 4, 8, 4]
n_attributes = [3, 3, 3, 4, 4, 5]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [16]:
datasets = ['(4,4)',]
n_values = [4,]
n_attributes = [4,]
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [102]:
datasets = ['(3,4)',]
n_values = [4,]
n_attributes = [3,]
n_epochs = 300
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [95]:
context_unaware = False # whether original or context_unaware simulations are evaluated
lazy = True # whether the lazy agent is used
if not lazy:
    if context_unaware:
        setting = 'context_unaware'
    else:
        setting = 'standard'
else:
    if context_unaware:
        setting = 'lazy_context_unaware'
    else:
        setting = 'lazy_context_aware'

### Determine vocab size

In [19]:
from collections import Counter

In [38]:
def loader(i,d,nr=0):
    # select first run
    path_to_run = paths[i] + '/' + str(setting) +'/' + str(nr) + '/'
    path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
    path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
    interaction = torch.load(path_to_interaction_train)
    print(path_to_interaction_train)
    return interaction

In [65]:
def retriever(i,d,interaction):
    sender_input = interaction.sender_input
    print(sender_input.shape)
    n_targets = int(sender_input.shape[1]/2)
    # get target objects and fixed vectors to re-construct concepts
    target_objects = sender_input[:, :n_targets]
    target_objects = k_hot_to_attributes(target_objects, n_values[i])
    # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
    (objects, fixed) = retrieve_concepts_sampling(target_objects, all_targets=True)
    concepts = list(zip(objects, fixed))

    # get distractor objects to re-construct context conditions
    distractor_objects = sender_input[:, n_targets:]
    distractor_objects = k_hot_to_attributes(distractor_objects, n_values[i])
    context_conds = retrieve_context_condition(objects, fixed, distractor_objects)
    
    return concepts, context_conds

In [24]:
def get_messages(interaction):
    messages = interaction.message.argmax(dim=-1)
    messages = [msg.tolist() for msg in messages]
    return messages

In [103]:
# go through all datasets
for i, d in enumerate(datasets):
    # select first run
    interaction = loader(i,d,0)
    
    messages = get_messages(interaction)
    all_symbols = [symbol for message in messages for symbol in message]
    symbol_counts = Counter(all_symbols)
    print(symbol_counts)
    print(len(symbol_counts))

results/(3,4)_game_size_10_vsf_3/lazy_context_aware/0/interactions/train/epoch_300/interaction_gpu0
Counter({0: 3876, 14: 366, 13: 363, 7: 359, 10: 303, 6: 275, 2: 260, 1: 251, 11: 246, 8: 217, 9: 207, 12: 141, 3: 134, 5: 127, 4: 103})
15


In [115]:
# calculate the average weighted message length weighted
messages = get_messages(interaction)
def get_length(messages):
    l = [0]*len(messages)
    for i,m in enumerate(messages):
        for w,s in enumerate(m):
            if s == 0.0:
                l[i] = w
                break
    return l

np.mean(get_length(messages))

1.5368013281682347

In [113]:
# average message length not weighted (each message only count once)
messages = get_messages(interaction)
messages_set = list(set([tuple(m) for m in messages]))
np.mean(get_length(messages_set))

1.670807453416149

### See which messages are produced for which concepts

In [21]:
datasets = ['(3,4)']
n_values = [4]
n_attributes = [3]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [67]:
# go through all datasets
for i, d in enumerate(datasets):
    # select first run
    interaction = loader(i,d)
    messages = get_messages(interaction)
    concepts, context_conds = retriever(i,d,interaction)

    # get random qualitative samples
    #fixed_index = random.randint(0, n_attributes[i]-1) # define a fixed index for the concept
    #n_fixed = random.randint(1, n_attributes[i]) # how many fixed attributes?
    n_fixed = 3
    #fixed_indices = random.sample(range(0, n_attributes[i]), k=n_fixed) # select which attributes are fixed
    fixed_indices = [0, 1, 2]
    #fixed_value = random.randint(0, n_values[i]-1) # define a fixed value for this index
    #fixed_values = random.choices(range(0, n_values[i]), k=n_fixed)
    fixed_values = [0, 0, 0]
    print(n_fixed, fixed_indices, fixed_values)
    #index_threshold = 20000 # optional: define some index threshold to make sure that examples are not taken from the beginning of training
    # TODO: adapt this loop such that multiple indices can be fixed
    all_for_this_concept = []
    for idx, (t_objects, t_fixed) in enumerate(concepts):
        #if sum(t_fixed) == 1 and t_fixed[fixed_index] == 1:# and idx > index_threshold:
        if sum(t_fixed) == n_fixed and all(t_fixed[fixed_index] == 1 for fixed_index in fixed_indices):
            for t_object in t_objects:
                if all(t_object[fixed_index] == fixed_values[j] for j, fixed_index in enumerate(fixed_indices)):
                    all_for_this_concept.append((idx, t_object, t_fixed, context_conds[idx], messages[idx]))
                    fixed = t_fixed
    if len(all_for_this_concept) > 0:
        #sample = random.sample(all_for_this_concept, 20)
        sample = all_for_this_concept
        column_names = ['game_nr', 'object', 'fixed indices', 'context condition', 'message']
        df = pd.DataFrame(sample, columns=column_names)
        path_to_save = 'analysis/quali_' + str(d) + '_' + str(setting) + '_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv'
        print(path_to_save)
        df.to_csv(path_to_save, index=False)
        print('saved ' + 'analysis/quali_' + str(d) + '_' + str(setting) + '_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv')
    else:
        raise ValueError("sample for dataset " + str(d) + " could not be generated")

results/(3,4)_game_size_10_vsf_3/lazy_context_aware/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([1807, 20, 12])
3 [0, 1, 2] [0, 0, 0]
analysis/quali_(3,4)_lazy_context_aware_[0. 0. 0.],[1. 1. 1.]all.csv
saved analysis/quali_(3,4)_lazy_context_aware_[0. 0. 0.],[1. 1. 1.]all.csv


In [52]:
# Groupby und Aggregation
def aggregate_messages(messages):
    message_counts = messages.value_counts().to_dict()
    unique_messages = list(message_counts.keys())
    frequencies = list(message_counts.values())
    return unique_messages, frequencies

def group_by_messages(df):
    grouped = df.groupby(['context condition','object','fixed indices'])['message'].apply(aggregate_messages).reset_index()
    grouped[['unique_messages', 'frequencies']] = pd.DataFrame(grouped['message'].tolist(), index=grouped.index)
    return grouped.drop('message', axis=1)

In [58]:
df = pd.read_csv('analysis/quali_(3,4)_lazy_context_aware_[0. 0. 0.],[1. 1. 1.]all.csv')
group_by_messages(df)

,context condition,object,fixed indices,unique_messages,frequencies
0,0,[0. 0. 0.],[1. 1. 1.],"[[2, 0, 13, 0], [2, 0, 0, 0]]","[60, 30]"
1,1,[0. 0. 0.],[1. 1. 1.],"[[2, 0, 13, 0], [2, 0, 0, 0]]","[50, 30]"
2,2,[0. 0. 0.],[1. 1. 1.],"[[2, 13, 0, 0]]",[100]


In [61]:
df = pd.read_csv('analysis/quali_(3,4)_lazy_context_aware_[2. 3. 3.],[1. 1. 1.]all.csv')
group_by_messages(df)

,context condition,object,fixed indices,unique_messages,frequencies
0,0,[2. 3. 3.],[1. 1. 1.],"[[6, 0, 0, 0]]",[80]
1,1,[2. 3. 3.],[1. 1. 1.],"[[6, 5, 0, 0], [6, 3, 0, 0], [6, 5, 3, 0], [5,...","[30, 20, 10, 10, 10]"
2,2,[2. 3. 3.],[1. 1. 1.],"[[6, 3, 0, 0]]",[90]


### Case study: concepts on different levels of abstraction

In [62]:
datasets = ['(3,4)']
n_values = [4]
n_attributes = [3]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [68]:
interaction = loader(0,datasets[0])
messages = get_messages(interaction)
concepts, context_conds = retriever(i,d,interaction)

results/(3,4)_game_size_10_vsf_3/lazy_context_aware/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([1807, 20, 12])


In [72]:
concept = np.array([0., 2., 0.,]) # Warum hat das Konzept 0 in der mitte fix
fixed = np.array([0., 1., 0.])
n_fixed = 1
fixed_indices = [1,]
fixed_values = [0, 1, 2]
print(n_fixed, fixed_indices, fixed_values)
#index_threshold = 20000 # optional: define some index threshold to make sure that examples are not taken from the beginning of training
# TODO: adapt this loop such that multiple indices can be fixed
all_for_this_concept = []
for idx, (t_objects, t_fixed) in enumerate(concepts):
    #if sum(t_fixed) == 1 and t_fixed[fixed_index] == 1:# and idx > index_threshold:
    if sum(t_fixed) == n_fixed and all(t_fixed[fixed_index] == 1 for fixed_index in fixed_indices):
        for t_object in t_objects:
            if all(t_object[fixed_index] == fixed_values[j] for j, fixed_index in enumerate(fixed_indices)):
                all_for_this_concept.append((idx, t_object, t_fixed, context_conds[idx], messages[idx]))
if len(all_for_this_concept) > 0:
    sample = random.sample(all_for_this_concept, 20)
    column_names = ['game_nr', 'object', 'fixed indices', 'context condition', 'message']
    df = pd.DataFrame(sample, columns=column_names)
    df.to_csv('analysis/quali_' + str(datasets[0]) + '_' + str(setting) + '_' + str(concept) + ',' + str(fixed) + '.csv', index=False)
    print('saved ' + 'analysis/quali_' + str(datasets[0]) + '_' + str(setting) + '_' + str(concept) + ',' + str(fixed) + '.csv')
else:
    raise ValueError("sample for dataset " + str(datasets[0]) + " could not be generated")

1 [1] [0, 1, 2]
saved analysis/quali_(3,4)_lazy_context_aware_[0. 1. 0.],[0. 1. 0.].csv


In [71]:
df = pd.read_csv('analysis/quali_(3,4)_lazy_context_aware_[0. 1. 2.],[1. 1. 1.].csv')
df

,game_nr,object,fixed indices,context condition,message
0,1045,[0. 1. 2.],[1. 1. 1.],1,"[3, 0, 0, 0]"
1,488,[0. 1. 2.],[1. 1. 1.],2,"[3, 9, 0, 0]"
2,1061,[0. 1. 2.],[1. 1. 1.],0,"[3, 0, 0, 0]"
3,1261,[0. 1. 2.],[1. 1. 1.],1,"[3, 0, 0, 0]"
4,335,[0. 1. 2.],[1. 1. 1.],2,"[3, 9, 0, 0]"
5,593,[0. 1. 2.],[1. 1. 1.],0,"[3, 0, 0, 0]"
6,1316,[0. 1. 2.],[1. 1. 1.],1,"[3, 0, 0, 0]"
7,1061,[0. 1. 2.],[1. 1. 1.],0,"[3, 0, 0, 0]"
8,151,[0. 1. 2.],[1. 1. 1.],0,"[3, 0, 0, 0]"
9,1046,[0. 1. 2.],[1. 1. 1.],2,"[3, 9, 0, 0]"


In [73]:
df = pd.read_csv('analysis/quali_(3,4)_lazy_context_aware_[0. 1. 0.],[0. 1. 0.].csv')
df # warum ist context condition hier immer 0??? Zeigt aber dass die Objekte alle mit der selben Nachricht benannt werden. Wie ist das für die anderen Attribute und andere Values. 

,game_nr,object,fixed indices,context condition,message
0,1794,[1. 0. 2.],[0. 1. 0.],0,"[14, 14, 0, 0]"
1,1514,[2. 0. 0.],[0. 1. 0.],0,"[14, 14, 0, 0]"
2,1276,[2. 0. 2.],[0. 1. 0.],0,"[14, 14, 0, 0]"
3,1514,[0. 0. 2.],[0. 1. 0.],0,"[14, 14, 0, 0]"
4,212,[2. 0. 2.],[0. 1. 0.],0,"[14, 14, 0, 0]"
5,212,[1. 0. 2.],[0. 1. 0.],0,"[14, 14, 0, 0]"
6,1276,[3. 0. 1.],[0. 1. 0.],0,"[14, 14, 0, 0]"
7,1021,[1. 0. 1.],[0. 1. 0.],0,"[14, 14, 0, 0]"
8,1794,[2. 0. 2.],[0. 1. 0.],0,"[14, 14, 0, 0]"
9,1276,[0. 0. 1.],[0. 1. 0.],0,"[14, 14, 0, 0]"


In [55]:
# same attribute value, different position -> different message
#df = pd.read_csv('analysis/quali_(3,4)_standard_1,1.csv')
#df

,game_nr,object,fixed indices,message
0,1644,[1. 1. 1.],[0. 1. 0.],"[11, 5, 5, 0]"
1,463,[2. 1. 1.],[0. 1. 0.],"[11, 5, 5, 0]"
2,1644,[2. 1. 3.],[0. 1. 0.],"[11, 5, 5, 0]"
3,463,[0. 1. 3.],[0. 1. 0.],"[11, 5, 5, 0]"
4,463,[0. 1. 1.],[0. 1. 0.],"[11, 5, 5, 0]"
5,1358,[1. 1. 0.],[0. 1. 0.],"[11, 5, 5, 0]"
6,1644,[3. 1. 0.],[0. 1. 0.],"[11, 5, 5, 0]"
7,1176,[3. 1. 3.],[0. 1. 0.],"[11, 5, 5, 0]"
8,268,[0. 1. 3.],[0. 1. 0.],"[11, 5, 5, 0]"
9,393,[0. 1. 2.],[0. 1. 0.],"[11, 5, 5, 0]"


In [56]:
#df = pd.read_csv('analysis/quali_(3,4)_standard_2,1.csv')
#df

,game_nr,object,fixed indices,message
0,1494,[2. 0. 1.],[0. 0. 1.],"[3, 3, 3, 0]"
1,951,[3. 0. 1.],[0. 0. 1.],"[3, 3, 2, 0]"
2,227,[1. 3. 1.],[0. 0. 1.],"[3, 2, 3, 0]"
3,1389,[0. 2. 1.],[0. 0. 1.],"[3, 3, 2, 0]"
4,1494,[0. 1. 1.],[0. 0. 1.],"[3, 3, 3, 0]"
5,194,[3. 2. 1.],[0. 0. 1.],"[3, 3, 2, 0]"
6,1389,[2. 3. 1.],[0. 0. 1.],"[3, 3, 2, 0]"
7,194,[1. 3. 1.],[0. 0. 1.],"[3, 3, 2, 0]"
8,951,[2. 1. 1.],[0. 0. 1.],"[3, 3, 2, 0]"
9,951,[0. 2. 1.],[0. 0. 1.],"[3, 3, 2, 0]"


### Dataset (3,4)

NOTE: Output contains redundant information: In the same game, the same message is used to refer to different objects because only one message is transmitted.

In [106]:
#df = pd.read_csv('analysis/quali_(3,4)_2,2.csv')
#df

,game_nr,object,fixed indices,message
0,761,[0. 1. 2.],[0. 0. 1.],"[11, 11, 11, 0]"
1,433,[2. 0. 2.],[0. 0. 1.],"[11, 11, 11, 0]"
2,433,[1. 0. 2.],[0. 0. 1.],"[11, 11, 11, 0]"
3,1211,[3. 0. 2.],[0. 0. 1.],"[11, 11, 11, 0]"
4,1522,[2. 0. 2.],[0. 0. 1.],"[11, 11, 11, 0]"
5,761,[1. 1. 2.],[0. 0. 1.],"[11, 11, 11, 0]"
6,1056,[2. 0. 2.],[0. 0. 1.],"[11, 11, 11, 0]"
7,184,[1. 2. 2.],[0. 0. 1.],"[11, 11, 11, 0]"
8,184,[0. 0. 2.],[0. 0. 1.],"[11, 11, 11, 0]"
9,1522,[0. 3. 2.],[0. 0. 1.],"[11, 11, 11, 0]"
